In [ ]:
# import necessary libraries
import pandas as pd
import os
import textstat
from openai import OpenAI
import json
import re
import requests
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# import prompts 
from jh_pfx_prompts import example, icd10_example, baseline_zeroshot_prompt, single_fewshot_prompt, multiple_fewshot_prompt, single_fewshot_icd10_labeling_prompt, multiple_fewshot_icd10_labeling_prompt

In [ ]:
# api key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
CLIENT = OpenAI(api_key = OPENAI_API_KEY)
OPENAI_MODEL = "gpt_4o"

In [ ]:
#reading levels
PROFESSIONAL = "Professional"
COLLEGE_GRADUATE = "College Graduate"
COLLEGE = "College"
TENTH_TO_TWELTH_GRADE = "10th to 12th grade"
EIGTH_TO_NINTH_GRADE = "8th to 9th grade"
SEVENTH_GRADE = "7th grade"
SIXTH_GRADE = "6th grade"
FIFTH_GRADE = "5th grade"
N_A = "N/A"

In [ ]:
# https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests#Flesch_reading_ease
def map_reading_level(flesch_reading_ease):
    if flesch_reading_ease < 10:
        return PROFESSIONAL
    elif 10.0 <= flesch_reading_ease < 30.0:
        return COLLEGE_GRADUATE
    elif 30.0 <= flesch_reading_ease < 50.0:
        return COLLEGE
    elif 50.0 <= flesch_reading_ease < 60.0:
        return TENTH_TO_TWELTH_GRADE
    elif 60.0 <= flesch_reading_ease < 70.0:
        return EIGTH_TO_NINTH_GRADE
    elif 70.0 <= flesch_reading_ease < 80.0:
        return SEVENTH_GRADE
    elif 80.0 <= flesch_reading_ease < 90.0:
        return SIXTH_GRADE
    elif 90.0 <= flesch_reading_ease < 100.0:
        return FIFTH_GRADE 
    else:
        return N_A

In [ ]:
# import fewshot examples
df_fewshot = pd.read.csv('pfx_fewshot_example_college.csv')

In [ ]:
# import evaluation data 
df_eval = pd.read.csv('pfx_evaluation_data.csv')

In [ ]:
# extract the json from openai
def extract_json(openai_response):
    if openai_response:
        json_match = re.search(r'```.*?(\{.*?\}).*?```', openai_response.message.content, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
            try:
                # converts extracted string to a json object 
                json_object = json.loads(json_str.replace('\n',''))
                return json_object
            except json.JSONDecodeError as e: #prints json decode error if there is one 
                print("JSON decoding failed: ", e)
                return dict()
        else:
            print("No JSON object found.")
            return dict()
    else:
        return None 

In [ ]:
# takes json object and generated icd10 labels for incidental findings 
def label_icd10s(pfx_outputs_json):
    pfx_icd10_fewshot_examples = ""
    for i, row in df_fewshot.iterrows():
        pfx_icd10_fewshot_examples += icd10_example.format(**row)
    pfx_icd10_codes = []
    pfx_icd10_response = None 
    for pfx_output in pfx_outputs_json :
        try:
            prompt = single_fewshot_icd10_labeling_prompt_fewshot_icd10_labeling_prompt.format(examples = pfx_icd10_fewshot_examples, PFx = pfx_output['PFx'])
        except:
            print("ERROR: %S" % pfx_output)
            return
        pfx_icd10_response = CLIENT.chat.completions.create(
            model = OPENAI_MODEL
            temperature = 0.0,
            messages = [
                {"role": "system", "content": "You are an ICD10 medical coder for incidental findings. "},
                {"role": "system", "content": prompt}
            ],
            stream = False,
        )
    pfx_icd10_codes.append(pfx_icd10_response.choices[0])
    return list(map(extract_json, pfx_icd10_codes))

In [ ]:
# gets zeroshot response 
pfx_zeroshot_outputs = []
for i, row in df_eval.iloc[:1].iterrows():
    prompt = baseline_zeroshot_prompt.format(Incidental_Finding = row["Incidental Finding"], Reading_Level = TENTH_TO_TWELTH_GRADE)
pfx_response = None 
pfx_response = CLIENT.chat.completions.create(
    model = OPENAI_MODEL,
    temperature = 0.0,
    messages = [
        {"role": "system", "content": "You are a md doctor rephrasing and explaining medical terminology to a patient in an understandable manor."}
        {"role": "system", "content": prompt}
    ],
    stream = False,
)

pfx_zeroshot_outputs.append(pfx_response.choices[0])

In [ ]:

# converts to a readable format 
pfx_zeroshot_outputs_json = list(map(extract_json, pfx_zeroshot_outputs))

In [ ]:
pfx_zeroshot_outputs_json

In [ ]:
pfx_zeroshot_output_df = pd.DataFrame(pfx_zeroshot_outputs_json)
pfx_zeroshot_outputs_icd10_labels = label_icd10s(pfx_zeroshot_outputs_json)

In [ ]:
pfx_zeroshot_output_df['_0_agent_icd10_codes'] = list(map(lambda x: list(x.values())[0] if x else "", pfx_zeroshot_outputs_icd10_labels))

In [ ]:
pfx_zeroshot_output_df["_0_icd10_matches"]= pfx_zeroshot_output_df.ICD10_code == pfx_zeroshot_output_df._0_agent_icd10_codes
pfx_zeroshot_output_df["_0_flesch"] = pfx_zeroshot_output_df['PFx'].apply(textstat.flesch_reading_ease)
pfx_zeroshot_output_df["_0_reading_level"] = pfx_zeroshot_output_df['_0_flesch'].apply(map_reading_level)
pfx_zeroshot_output_df["_0_reading_level_matches"] = pfx_zeroshot_output_df['_0_reading_level'] == TENTH_TO_TWELTH_GRADE

In [ ]:
sum(pfx_zeroshot_output_df["_0_icd10_matches"])/len(pfx_zeroshot_output_df.index)

In [ ]:
pfx_zeroshot_output_df[pfx_zeroshot_output_df._0_reading_level != TENTH_TO_TWELTH_GRADE].count()

In [ ]:
# gets single fewshot response 
pfx_fewshot_examples = ""
for i, row im df_fewshot.iterrows():
    pfx_fewshot_examples += example.format(**row)
pfx_fewshot_outputs = []
pfx_response = None
for i, row in df_eval.iloc[:1].iterrows():
    prompt = single_fewshot_prompt.format(examples = pfx_fewshot_examples, Incidental_Finding = row["Incidental_Finding"], Reading_Level = TENTH_TO_TWELTH_GRADE)
pfx_response = CLIENT.chat.completions.create(
    model = OPENAI_MODEL,
    temperature = 0.0
    messages = [
        {"role": "system", "content": "You are a md doctor rephrasing and explaining medical terminology to a patient in an understandable manor."}
        {"role": "system", "content": prompt}
    ],
    stream = False,
)
pfx_fewshot_outputs.append(pfx_response.choices[0])

In [ ]:
pfx_fewshot_outputs_json = list(map(extract_json, pfx_fewshot_outputs))

In [ ]:
pfx_fewshot_outputs_json

In [ ]:
pfx_fewshot_output_df = pd.DataFrame(pfx_fewshot_outputs_json)
pfx_fewshot_outputs_icd10_labels = label_icd10s(pfx_fewshot_outputs_json)

In [ ]:
pfx_fewshot_output_df['_0_agent_icd10_codes'] = list(map(lambda x: list(x.values())[0] if x else "",pfx_fewshot_outputs_icd10_labels))
pfx_fewshot_output_df["_0_icd10_matches"]= pfx_fewshot_output_df.ICD10_code == pfx_fewshot_output_df._0_agent_icd10_codes
pfx_fewshot_output_df["_0_flesch"] = pfx_fewshot_output_df['PFx'].apply(textstat.flesch_reading_ease)
pfx_fewshot_output_df["_0_reading_level"] = pfx_fewshot_output_df['_0_flesch'].apply(map_reading_level)

In [ ]:
sum(pfx_fewshot_output_df["_0_icd10_matches"])/len(pfx_fewshot_output_df.index)

In [ ]:
pfx_fewshot_output_df[pfx_fewshot_output_df._0_reading_level != TENTH_TO_TWELTH_GRADE].count()

In [ ]:
pfx_fewshot_output_df[pfx_fewshot_output_df._0_flesch < 60].count()